In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

data = pd.read_csv("data.csv")
ticket = pd.read_csv("maintenance.csv")
print(data.columns)
ticket.columns = pd.Series(ticket.columns).apply(lambda x : x.strip())
print(ticket.columns)
data["TermId"].value_counts()

In [ ]:
data["FaultStartTime"] = pd.to_datetime(data["FaultStartTime"], format='%Y-%m-%d %H:%M:%S.%f')
data["FaultEndTime"] = pd.to_datetime(data["FaultEndTime"], format='%Y-%m-%d %H:%M:%S.%f')

ticket["Ticket start"] = pd.to_datetime(ticket["Ticket start"], format='%d-%m-%Y %H:%M')
ticket["Ticket stop"] = pd.to_datetime(ticket["Ticket stop"], format='%d-%m-%Y %H:%M')

data = data.sort_values(by = ["FaultStartTime"], ignore_index = True)
ticket = ticket.sort_values(by = ["Ticket start"], ignore_index = True)

data = data[data["FaultStartTime"] >= dt.datetime(2020, 8, 1, 0, 0, 0)]
data = data[data["FaultStartTime"] < dt.datetime(2020, 9, 1, 0, 0, 0)]
data.reset_index()

In [ ]:
data["FaultDesc"].unique()

In [ ]:
errortypes = data["FaultDesc"].unique().tolist()
print(len(errortypes))
errors = {errortypes[i] : "e{}".format(i+1) for i in range(len(errortypes))}
data = data.replace({"FaultDesc" : errors})

In [ ]:
atm_id = {}
atm = data["TermId"].unique().tolist()
for i in atm:
    atm_id[i] = data[data["TermId"] == i][['FaultStartTime', 'FaultEndTime', 'FaultDesc']].values.tolist()

atm_id

In [ ]:
dataset = pd.DataFrame() 
for name in list(atm_id.keys()):
    
    sub_data = np.array(atm_id[name])
    month_start = sub_data[0,0].date()#dt.datetime(2020, 8, 1, 0, 0, 0)
    month_start = dt.datetime(month_start.year, month_start.month, month_start.day, 0, 0, 0)
    month_end = dt.datetime(2020, 8, 31, 23, 59, 59)
    x = 2
    m = 10
    y = 6
    date_index = 0 #index of fault start date in list
    mu_len = dt.timedelta(days = m, hours = 0, minutes = 0, seconds = 0)  # length of each measurement unit
    ow_gap = dt.timedelta(days = 2, hours = 0, minutes = 0, seconds = 0)  # gap between two successive observation windows
    ow_start = month_start
    pred_point = ow_start + dt.timedelta(days = x*m, hours = 0, minutes = 0, seconds = 0)
    pred_end = pred_point + dt.timedelta(days = y, hours = 0, minutes = 0, seconds = 0)

    error_count = {e : 0 for e in errors.values()}
    rows = []
    target = []

    while(pred_end <= month_end):
        mu_start = ow_start
        mu_end = mu_start + mu_len
        single_row = []
        colnames = []
    
        for mu_no in range(x):
            s = sub_data[sub_data[:,date_index] >= mu_start]
            error_mu = list(s[s[:,date_index] < mu_end][:,2])
            for e in error_mu:
                error_count[e] += 1 
            mu_start = mu_end
            mu_end = mu_end + mu_len
            single_row.extend(list(error_count.values()))
            error_count = {e : 0 for e in errors.values()} 
            for e in errors.values():
                colnames.append(e + ".m{}".format(mu_no + 1))
       
        s = sub_data[sub_data[:,date_index] >= ow_start]
        s = s[s[:,date_index] < pred_point]
            
        for e in errors.values():
            v = [] 
            dist_from_pred = pd.Series(pred_point - (s[s[:,2] == e][:,date_index])).apply(lambda x : (x.seconds)/60)
            if dist_from_pred.shape[0] != 0:
                single_row.extend([dist_from_pred.min(),dist_from_pred.max(),dist_from_pred.mean()])
            #    dist_from_pred = list(dist_from_pred)
                dist_from_pred = list(dist_from_pred.sort_values(ascending = False))
                for i in range(len(dist_from_pred)-1):
                    v.append(dist_from_pred[i] - dist_from_pred[(i+1)])
                if len(v) == 0:
                    single_row.extend([0,0])
                else:
                    single_row.extend([np.mean(v), np.std(v)])
            else:
                single_row.extend([0,0,0,0,0])   
                
            colnames.extend(["min(D{})".format(e[1:]),"max(D{})".format(e[1:]),"mean(D{})".format(e[1:]),"mean(V{})".format(e[1:]),"std(V{})".format(e[1:])])
    
        t = ticket[ticket["ATM ID"] == name]
        t = t[t["Ticket start"] <= pred_end]
        t = t[t["Ticket start"] >= mu_start]
        #t = t[t["Activity comment"].str.contains("SLM")]
        target.append(0 if t.shape[0] == 0 else 1)

        rows.append(single_row)
        
        ow_start = ow_start + ow_gap
        pred_point = ow_start + dt.timedelta(days = x*m, hours = 0, minutes = 0, seconds = 0)
        pred_end = pred_point + dt.timedelta(days = y, hours = 0, minutes = 0, seconds = 0)    
        
    profile = pd.DataFrame([name for i in range(len(rows))], columns = ["ATM ID"])
    df = pd.DataFrame(rows, columns = colnames)
    target_df = pd.DataFrame(target, columns = ["Target"])
    df = pd.concat([profile, df, target_df], axis = 1)
    
    dataset = pd.concat([dataset, df])    


In [ ]:
dataset

In [ ]:
from collections import Counter
Counter(dataset.Target)

In [ ]:
dataset.to_csv("Feature.csv", index = False)